In [2]:
import torch
import torch.nn.functional as F
import torchvision
import numpy as np
import fashion

In [3]:
batch_size = 50

train_loader = torch.utils.data.DataLoader(
    fashion.MNIST_fashion('fashion', train=True, download=True, transform=torchvision.transforms.ToTensor()))
test_loader = torch.utils.data.DataLoader(
    fashion.MNIST_fashion('fashion', train=True, download=True, transform=torchvision.transforms.ToTensor()))

Processing...
Done!


In [4]:
class MnistModel(torch.nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, 5, padding=2, bias=True)
        self.maxP1 = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(32, 64, 5, padding=2, bias=True)
        self.maxP2 = torch.nn.MaxPool2d(2, 2)
        self.fc1 = torch.nn.Linear(64*7*7, 1024)
        self.drop1 = torch.nn.Dropout(0.5)
        self.fc2 = torch.nn.Linear(1024, 10)
    
    def forward(self, x):
        x = self.maxP1(F.relu(self.conv1(x)))
        x = self.maxP2(F.relu(self.conv2(x)))
        x = x.view(-1, 64*7*7)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = self.fc2(x)
        return x

In [5]:
model = MnistModel().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [6]:
model.train()
num_epochs = 20000
epoch = 0

for epoch in range(0, num_epochs, len(train_loader)):
    for X, y in train_loader:
        X, y = torch.autograd.Variable(X).cuda(), torch.autograd.Variable(y).cuda()
        optimizer.zero_grad()
        output = model(X)
        loss = F.cross_entropy(output, y)
        loss.backward()
        optimizer.step()
        predictions = output.data.max(1)[1]
    if epoch%1000==0:
        accuracy = predictions.eq(y.data).sum()/batch_size
        print("step {}, accuracy: {:.2f}".format(epoch, accuracy))

model.eval()
correct = 0
for data, target in test_loader:
    data, target = torch.autograd.Variable(data, volatile=True).cuda(), torch.autograd.Variable(target).cuda()
    output = model(data)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()

print('\nTest set: Accuracy: {:.2f}%'.format(100. * correct / len(test_loader.dataset)))

step 0, accuracy: 0.02

Test set: Accuracy: 89.91%


In [12]:
Y_train.shape[0]

60000